In [5]:
from backtesting import Backtest
import inspect
import pandas as pd
import numpy as np
import seaborn as sns
from itertools import combinations
import tensorflow as tf
import scipy.stats as ss
import strats, get_data
from typing import Tuple
import time
from sklearn.model_selection import BaseCrossValidator
import matplotlib.pyplot as plt
from datetime import date, datetime
import base64
from io import BytesIO

In [6]:
stock_obj = get_data.yFinData("SPY")
try:
    ydata = stock_obj.get_ohlcv()
except:
    print('Uable to download data.')

In [8]:
def get_back_test_comparasion(ydata: pd.DataFrame, strategy: str, data_range, strategy_params: dict,
                              cash: int=1_000_000, commission: float=0.):
    """
    backtest vs buy/hold strategy in strats.py
    input: stock OHLCV dataframe
    output: dataframe of strategy returns, dictionary of trades and equity curve
    """
    avail_strats = [obj for name, obj in inspect.getmembers(strats, inspect.isclass)
                    if (obj.__name__ == strategy) or (obj.__name__ == "BuyAndHold")]
    if not data_range.isdecimal():
        corresponding = {"6m":0.5, "1y":1., "2y":2.}
        data = ydata.iloc[-int(float(corresponding[data_range])*252):]
    else:
        data = ydata.loc["{}-12-31".format(int(data_range)-1):"{}-12-31".format(int(data_range)),]

    temp = []
    sname_temp = []
    equity_trades = {}
    for s in avail_strats:
        if data.shape[0] == 0:
            continue

        bt = Backtest(data, s, cash=cash, commission=commission)
        if s.__name__ == 'SmaCross':
            stats = bt.run(slow = strategy_params['sma_slow'],
                           fast = strategy_params['sma_fast'],
                           long_only = strategy_params['long_only'])
        elif s.__name__ == 'MacdSignal':
            stats = bt.run(fastperiod = strategy_params['fast_period'],
                           slowperiod = strategy_params['slow_period'],
                           signalperiod = strategy_params['signal_period'],
                           long_only = strategy_params['long_only'])
        elif s.__name__ == 'StochOsci':
            stats = bt.run(fastk_period = strategy_params['fast_k_period'],
                           slowk_period = strategy_params['slow_k_period'],
                           slowd_period = strategy_params['slow_d_period'],
                           overbought = strategy_params['overbought'],
                           oversold = strategy_params['oversold'],
                           long_only = strategy_params['long_only'])
        elif s.__name__ == 'StochRsi':
            stats = bt.run(timeperiod = strategy_params['time_period'],
                           fastk_period = strategy_params['fast_k_period'],
                           fastd_period = strategy_params['fast_d_period'],
                           overbought = strategy_params['overbought'],
                           oversold = strategy_params['oversold'],
                           long_only = strategy_params['long_only'])
        else:
            stats = bt.run()
        sname = str(stats["_strategy"])
        sname_temp.append("{}_{}".format(sname, data_range))
        temp.append(stats[:27])
        equity_trades["{}_{}".format(sname, data_range)] = (stats["_equity_curve"], stats["_trades"])

    strat_returns = pd.concat(temp, axis=1)
    strat_returns.columns = sname_temp
    return strat_returns

In [9]:
df = get_back_test_comparasion(ydata, "SmaCross", "2y",
                               {"sma_slow":5, "sma_fast":10, "long_only":15}, 1_000_000, 0.)

In [10]:
df

,BuyAndHold_2y,"SmaCross(slow=5,fast=10,long_only=15)_2y"
Start,2019-04-03 00:00:00,2019-04-03 00:00:00
End,2021-04-01 00:00:00,2021-04-01 00:00:00
Duration,729 days 00:00:00,729 days 00:00:00
Exposure Time [%],99.603175,38.293651
Equity Final [$],1442599.071198,1048507.105701
Equity Peak [$],1442599.071198,1050204.424549
Return [%],44.259907,4.850711
Buy & Hold Return [%],45.014738,45.014738
Return (Ann.) [%],20.108246,2.396636
Volatility (Ann.) [%],30.987535,21.858675


In [166]:
#!pip install altair
import altair as alt

## Transform the dataframe for easier charting
data = df.transpose(copy=True)
data

## Shorten the names of the strategies and defin the metrics we want to compare
strategies = ["Buy & Hold", "SMA Cross"]
metrics = ["Return (Ann.) [%]", "Exposure Time [%]", "Volatility (Ann.) [%]", 
           "Max. Drawdown [%]", "Sharpe Ratio", "Sortino Ratio", "Calmar Ratio"]

## Set width and height for the sub-charts
subchart_width = 60
subchart_height = 200
subcharts = list()

for i in range(len(metrics)):
    ## Get the metric for each subchart
    source = pd.DataFrame({
    'Strategy': strategies,
    'Metric': data[metrics[i]]})
    
    ## Make the Altair chart
    subchart = alt.Chart(source).mark_bar().encode(
        alt.X('Strategy', 
              axis=alt.Axis(title = '')
             ),
        alt.Y('Metric', 
              axis=alt.Axis(title = metrics[i])
             ),
        tooltip='Metric',
        color='Strategy'
    ).properties(
        width=subchart_width,
        height=subchart_height
    )
    subcharts.append(subchart)
    
chart = (subcharts[0] | subcharts[1] | subcharts[2] | subcharts[3] | subcharts[4] | subcharts[5] | subcharts[6])

chart

alt.HConcatChart(...)

In [167]:
chart.save('comparison.html')